In [1]:
import sys
sys.path.append('../ames') # path the the directory

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.api import OLS
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import re
import itertools
%matplotlib inline 

In [2]:
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression

In [3]:
import config
from data_prep import clean, add_features

In [4]:
housing_initial = clean(config.HOUSING_CSV)
housing = housing_initial.copy()
housing = add_features(housing)

In [5]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
ridge = Ridge()
lasso = Lasso()
net   = ElasticNet()

In [11]:
lin_reg_cols = ['Toilets','WoodDeckSF','TotRmsAbvGrd','BedroomAbvGr','Showers',
                'FirePlaces','YearBuilt','YearRemodAdd',
                'TotalBsmtSF','DecadeBuilt'
               ]

cols_to_dummy = ['Foundation','RoofStyle','SaleCondition',
                 'Neighborhood','Electrical','HouseStyle','SaleType','MSSubClass',
                 'LandContour','PavedDrive','GarageFinish','MSZoning'
                ]

dummy_suffix = ['Qual','Cond','Type']
for col in housing.columns:
    if col not in cols_to_dummy and col not in lin_reg_cols:
        for suff in dummy_suffix:
            #print(re.search(r'%s$'%dummy_suffix, col))
            if re.search(r'%s$'%suff, col) is not None:
                cols_to_dummy.append(col)
                break

In [12]:
ridge_df = housing.copy()
ridge_df = housing[['Neighborhood','OverallQual','BedroomAbvGr','TotalBsmtSF','DecadeBuilt','TotRmsAbvGrd','KitchenQual']]
prices = housing['SalePrice']

dummy_cols = []
lin_cols = []
for col in ridge_df.columns:
    if col in cols_to_dummy:
        dummy_cols.append(col)
    else:
        lin_cols.append(col)

ridge_df = ridge_df.fillna(0.0)
ridge_df = ridge_df.filter(items=lin_cols+dummy_cols)
if len(dummy_cols)>0:
    ridge_df = pd.get_dummies(ridge_df,columns=dummy_cols,drop_first=True)

In [13]:
coefs = []
intercepts = []
R2  = []
alphaRange = [.0005,.0001,.01,1,5]
for alpha in alphaRange:
    ridge.set_params(alpha=alpha)  
    ridge.fit(ridge_df,prices)
    intercepts.append(ridge.intercept_)
    coefs.append(ridge.coef_)
    R2.append(ridge.score(ridge_df,prices)) 

In [14]:
print(R2)

[0.836537310525765, 0.8365373129484573, 0.8365363510320292, 0.8353683629229732, 0.8315730565217787]
